In [7]:
# Import relevant libralies
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import os

In [9]:
#Load gisette
gisette_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/gisette/"
gisette_train_x = pd.read_csv(os.path.join(gisette_path, 'GISETTE/gisette_train.data'),
                         delim_whitespace=True,
                         header = None)
gisette_train_y = pd.read_csv(os.path.join(gisette_path, 'GISETTE/gisette_train.labels'),
                         delim_whitespace=True,
                         header = None)
gisette_test_x = pd.read_csv(os.path.join(gisette_path, 'GISETTE/gisette_valid.data'),
                         delim_whitespace=True,
                         header = None)
gisette_test_y = pd.read_csv(os.path.join(gisette_path, 'gissete_valid.labels'),
                         delim_whitespace=True,
                         header = None)


KeyboardInterrupt: 

# Homework 3
Author: Mao Nishino

## Problem 1
Implement the Logistic Regression learning by gradient ascent as described in class.
Before using logistic regression, be sure to normalize the variables of the training set
to have zero mean and standard deviation 1, and to use the exact same transformation
on the test set, using the mean and standard deviation of the training set.
### Problem (a)
Using the Gisette data, train a logistic regressor on the training set, starting
with $w^{(0)} = 0$, with 300 gradient ascent iterations and shrinkage $λ = 0.0001$ in
the update equation:

$\displaystyle w^{(t+1)} \leftarrow w^{(t)} - \eta \lambda w^{(t)} + \frac{\eta}{N} \frac{\partial}{\partial w} L(w^{(t)})$

where $L(w^{(t)})$ s the log likelihood from page 7 of the Logistic Regression
slides. Observe that there is an extra factor of 1/N in the loss term compared to
the class notes.
Find a good learning rate η such that the log-likelihood converges in at most 300
iterations and is monotonically increasing. Plot the training log-likelihood vs
iteration number. Report in a table the misclassification error on the training and
test set. On the same graph, plot the Receiver Operating Characteristic (ROC)
curve of the obtained model on the training and test set.